In [36]:
import json
import pandas as pd
import requests
#import string

def remove_hex(s):
    '''
    Method to remove hex characters from string. Note that import string has to be called here, and not above.
    '''    
    import string
    printable = set(string.printable)
    s2 = ''.join(filter(lambda x: x in printable, s))
    return(s2)

def remove_chars(s):
    '''
    Method to remove punctuations and other non-unicode characters.
    '''
    to_be_removed = ['translated_text','annotation_list',':','{','}','\n','[',']','  ','&','?','!',';','.','*','(',')','^','+','=','\"','\'','>','<','-','\\']
    import codecs
    
    s = codecs.decode(s, 'unicode_escape')
    
    s = remove_hex(s)
    #print("translated_text" in s)
    for char in to_be_removed:
        s = s.replace(char,'')
    return s

                     
#Read the corrupt JSON file -- lots of issues: (A) Unicode decoding difficult (B) Various invalid characters
def main():

    f = open('fear_speech_data.json','rb')
    fout2 = open('fear_speech.csv','w')
                     
    flag = 0 #Flag to understand which tag encountered last
    string1 = '' #Strings to keep only required text (tags)
    string2 = '' #Strings to keep only required text (tags)
    current=''
    c=0

    for line in f.readlines():
        line = line.decode("utf-8",errors="ignore")
        line=str(line)

        l = line.strip().split(":")
        #Tag is translated text. It has to be written.
        if "translated_text" in l[0]: #or "annotation_list" in l[0]:
            current='tt' #current saves the tag encountered currently
            flag = 0
            string1+=line+'\t'
        #Tag is annotation_list (ground truth labels). It has to be written.
        elif "annotation_list" in l[0]:
            flag=0
            current='al'
            string2 += line+'\t'
        #Message text to be ignored. Flag set to 1.
        elif "message_text" in l[0] or flag==1:
            flag = 1
            current = 'mt'
            continue
        #Propagation text to be ignored. Flag set to 1.
        #However, here we write to the file.
        elif "propagation" in l[0]:
            flag = 1
            current='pp'
            string3 = remove_chars(string1+string2)
            c+=1
            fout2.write(string3+'\n')
            current = ''
            string1 = ''
            string2 = ''
            #if c>=2:
            #    break
        #If the tag is translated_text or annotation_list, append the string to the line. This line is to be written.
        elif flag==0 and current=='tt':
            string1+=line
        elif flag==0 and current=='al':
            string2+=line
    fout2.close()
    print("Writing completed.")

if __name__ == "__main__":
    main()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: invalid escape sequence '\@'
/usr/lib/python3/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: invalid escape sequence '\<'
/usr/lib/python3/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: invalid escape sequence '\G'
/usr/lib/python3/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: invalid escape sequence '\ '


Writing completed.
